In [1]:
# !pip install nltk
# !pip install num2words
# !pip install sklearn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

# !pip install scikit-learn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
INPUT_DATA_FOLDER = r'C:\\Users\\Cyberpower\\python_course\\venv\\Scripts\\studium\\uni\\am_python\\hw2'
DATA_FILE = 'isw_all_days_parsed.csv'

# OUTPUT_FOLDER = r'C:\Users\Cyberpower\python_course\venv\Scripts\studium\py_project'
OUTPUT_DATA_FILE = 'isw_all_days_parsed_v1.csv'
os.getcwd()

'C:\\Users\\Cyberpower\\python_course\\venv\\Scripts\\studium\\uni\\am_python\\hw2'

In [3]:
df = pd.read_csv(f'{DATA_FILE}', sep = ';')

In [4]:
def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_stop_words = {"no","not"}
    stop_words = stop_words - stop_stop_words
    
    words = word_tokenize(str(data))
    
    new_text=''
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [5]:
def remove_one_letter_word(data):
    words = word_tokenize(str(data))
    
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    
    return new_text

In [6]:
def convert_lower_case(data):
    return np.char.lower(data)

In [7]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [8]:
def remove_punctuation(data):
    symbols = "!\"#$%&()+*—.-/:;<=>?@[\]^_`{|}~\n"
    
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, '  ', ' ')
    data = np.char.replace(data, ',', '')
    return data

In [9]:
def stemming(data):
    stemmer = PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def lemmatizing(data):
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + lemmatizer.lemmatize(w)
    return new_text

In [10]:
def conver_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        if w.isdigit():
            if int(w)<1000000000000:
                w = num2words(w)
            else:
                w=''
        new_text = new_text + ' ' + w
    new_text = np.char.replace(new_text,'-',' ')
    
    return new_text

In [11]:
def remove_url_from_string(data):
    words = word_tokenize(str(data))
    
    new_text = ''
    for w in words:
        w = re.sub(r'^https?:\/\/.*[\r\n]*', '', str(w), flags = re.MULTILINE)
        w = re.sub(r'^http?:\/\/.*[\r\n]*', '', str(w), flags = re.MULTILINE)
        
        new_text = new_text + ' ' + w
        
    return new_text

In [12]:
def preprocess(data, word_root_algo):
    data = remove_one_letter_word(data)
    data = remove_url_from_string(data)
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = conver_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = conver_numbers(data)
    
    if word_root_algo == "lemm":
#         print('lemm')
        data = lemmatizing(data)
    else:
#         print('stem')
        data = stemming(data)
        
    data = remove_punctuation(data)
    data = remove_stop_words(data)
    
    return data
# print(1)

In [13]:
df['report_text_lemm']= df['main_html_v7'].apply(lambda x: preprocess(x,'lemm'))
df['report_text_stemm']= df['main_html_v7'].apply(lambda x: preprocess(x,'stemm'))

In [14]:
df

,date,short_url,title,text_title,full_url,main_html,main_html_v2,main_html_v7,report_text_lemm,report_text_stemm
0,2022-02-24,russia_ukraine_warning_update_initial_russian_...,Russia-Ukraine Warning Update: Initial Russian...,Russia-Ukraine Warning Update: Initial Russian...,/backgrounder/russia-ukraine-warning-update-in...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna S...",\nRussian President Vladimir Putin began a lar...,russian presid vladimir putin began larg scal...,russian presid vladimir putin began larg scal...
1,2022-02-25,russia_ukraine_warning_update_russian_offensiv...,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step...",\nRussian forces entered the outskirts of Kyiv...,russian forc enter outskirt kyiv west bank dn...,russian forc enter outskirt kyiv west bank dn...
2,2022-02-27,russia_ukraine_warning_update_russian_offensiv...,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna St...",\nThe Russian military has likely recognized t...,russian militari like recogn initi expect lim...,russian militari like recogn initi expect lim...
3,2022-02-28,russian_offensive_campaign_assessment_february...,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna S...",\nThe Russian military is reorganizing its mil...,russian militari reorgan militari effort atte...,russian militari reorgan militari effort atte...
4,2022-03-01,russian_offensive_campaign_assessment_march_1,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, March 1",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nFrederick W. Kagan, George Barros, and Kat...",\nRussian forces are completing the reinforcem...,russian forc complet reinforc resuppli troop ...,russian forc complet reinforc resuppli troop ...
...,...,...,...,...,...,...,...,...,...,...
329,2023-01-21,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, January...","Russian Offensive Campaign Assessment, January...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nKarolina Hird, Grace Mappes, Angela Howard...",\nThe Ukrainian defense of Bakhmut is likely a...,ukrainian defens bakhmut like strateg sound e...,ukrainian defen bakhmut like strateg sound ef...
330,2023-01-22,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, Januar...","Russian Offensive Campaign Assessment, Januar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nRussian Offensive Campaign Assessment, Janu...",\nISW is publishing an abbreviated campaign up...,isw publish abbrevi campaign updat today janu...,isw publish abbrevi campaign updat today janu...
331,2023-01-23,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, January...","Russian Offensive Campaign Assessment, January...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nRussian Offensive Campaign Assessment, Janu...",\nUkrainian intelligence assessed that Russian...,ukrainian intellig ass russian forc prepar of...,ukrainian intellig assess russian forc prepar...
332,2023-01-24,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, Janua

In [15]:
df.shape
df.iloc[1,7]

'\nRussian forces entered the outskirts of Kyiv on the west bank of the Dnipro on February 25. Russian sabotage groups in civilian clothes are reportedly active in downtown Kyiv.\nRussian forces have so far failed to enter Kyiv’s eastern outskirts. Ukrainian forces have successfully slowed Russian troops, which have temporarily abandoned the failed attempt to take the city of Chernihiv and are instead bypassing it.\nElements of the Russian 76th VDV (Airborne) division have concentrated in southeastern Belarus likely for use along the Chernihiv-bypass axis toward Kyiv in the next 24 hours.\nRussian forces will likely envelop Kharkhiv in the next 24 hours after failing to enter the city through frontal assaults on February 24.\nRussian forces have achieved little success on frontal assaults or envelopments against Ukrainian forces in Donbas but may not have intended to do more than pin Ukrainian forces in the east.\nNorth of Crimea, Russian forces fully captured Kherson and are likely on

In [16]:
df.to_csv(f"{OUTPUT_DATA_FILE}", sep=";", index=False)

In [17]:
docs = df['report_text_lemm'].tolist()
# docs

In [18]:
string = ""
cv = CountVectorizer(max_df=0.98,min_df=2)
word_count_vector = cv.fit_transform(docs)
word_count_vector.shape
word_count_vector

<334x6531 sparse matrix of type '<class 'numpy.int64'>'
	with 197579 stored elements in Compressed Sparse Row format>

In [19]:
import scipy.sparse
arr_names = cv.get_feature_names_out(input_features='array-like')
# cv.fit_transform?
# arr = cv.fit_transform(df['report_text_stemm']).toarray()
# with open('count_vectorizer_v11.pkl', 'wb') as handle:
#     pickle.dump(cv ,handle)
    
handle = open("count_vectorizer_v11.pickle", 'wb') #wb= write in bytes. 
pickle.dump(cv, handle) #use pickle to dump the grid3 we trained, as 'Tfidf_LogR.pickle' in wb format
handle.close() 


In [20]:
tfidf_transformer =  TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)
# tfidf_transformer

TfidfTransformer()

In [21]:
with open('tfidf_transformer_v1.pkl', 'wb') as handle:
    pickle.dump(tfidf_transformer, handle)

In [22]:
df_idf = pd.DataFrame(tfidf_transformer.idf_, index = cv.get_feature_names_out(), columns = ['idf_weights'])
df_idf.sort_values(by = ['idf_weights'])

,idf_weights
effort,1.021117
citi,1.021117
ten,1.021117
eleven,1.021117
gener,1.024170
...,...
motion,5.715518
motorcycl,5.715518
motuzyanuk,5.715518
molochna,5.715518


In [23]:
tf_idf_vector = tfidf_transformer.transform(word_count_vector)

In [31]:
tfidf = pickle.load(open('tfidf_transformer_v1.pkl', 'rb'))
cv = pickle.load(open('count_vectorizer_v11.pickle', 'rb'))
feature_names = cv.get_feature_names_out()
feature_names

array(['10', '100', '100km', ..., 'zyuganov', 'коридор', 'сухопутный'],
      dtype=object)

In [32]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key = lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn = 10):
    '''get the feature names and tf-idf score of top n items '''
    
    sorted_items = sorted_items[:topn]
    
    score_vals = []
    feature_vals = []
    
    for idx, score in sorted_items:
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
        
    results = {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]] = score_vals[idx]
    return results

In [34]:
def convert_doc_to_vector(doc):
    feature_names = cv.get_feature_names_out()
    top_n = 100
    tf_idf_vector = tfidf.transform(cv.transform([doc]))
    
    sorted_items = sort_coo(tf_idf_vector.tocoo())
    
    keywords = extract_topn_from_vector(feature_names, sorted_items,top_n)
    
    return keywords

In [35]:
df['keywords'] = df['report_text_lemm'].apply(lambda x: convert_doc_to_vector(x))

In [36]:
# df.iloc[1,10]
# df.iloc[333,10]
df

,date,short_url,title,text_title,full_url,main_html,main_html_v2,main_html_v7,report_text_lemm,report_text_stemm,keywords
0,2022-02-24,russia_ukraine_warning_update_initial_russian_...,Russia-Ukraine Warning Update: Initial Russian...,Russia-Ukraine Warning Update: Initial Russian...,/backgrounder/russia-ukraine-warning-update-in...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna S...",\nRussian President Vladimir Putin began a lar...,russian presid vladimir putin began larg scal...,russian presid vladimir putin began larg scal...,"{'pm': 0.388, 'airport': 0.262, 'kyiv': 0.244,..."
1,2022-02-25,russia_ukraine_warning_update_russian_offensiv...,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","Mason Clark, George Barros, and Kateryna Step...",\nRussian forces entered the outskirts of Kyiv...,russian forc enter outskirt kyiv west bank dn...,russian forc enter outskirt kyiv west bank dn...,"{'pm': 0.36, 'zero': 0.351, 'februari': 0.336,..."
2,2022-02-27,russia_ukraine_warning_update_russian_offensiv...,Russia-Ukraine Warning Update: Russian Offensi...,Russia-Ukraine Warning Update: Russian Offensi...,/backgrounder/russia-ukraine-warning-update-ru...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna St...",\nThe Russian military has likely recognized t...,russian militari like recogn initi expect lim...,russian militari like recogn initi expect lim...,"{'februari': 0.553, 'kyiv': 0.262, 'seven': 0...."
3,2022-02-28,russian_offensive_campaign_assessment_february...,"Russian Offensive Campaign Assessment, Februar...","Russian Offensive Campaign Assessment, Februar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nMason Clark, George Barros, and Kateryna S...",\nThe Russian military is reorganizing its mil...,russian militari reorgan militari effort atte...,russian militari reorgan militari effort atte...,"{'februari': 0.593, 'eight': 0.267, 'kyiv': 0...."
4,2022-03-01,russian_offensive_campaign_assessment_march_1,"Russian Offensive Campaign Assessment, March 1...","Russian Offensive Campaign Assessment, March 1",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nFrederick W. Kagan, George Barros, and Kat...",\nRussian forces are completing the reinforcem...,russian forc complet reinforc resuppli troop ...,russian forc complet reinforc resuppli troop ...,"{'kyiv': 0.357, 'envelop': 0.287, 'chernihiv':..."
...,...,...,...,...,...,...,...,...,...,...,...
329,2023-01-21,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, January...","Russian Offensive Campaign Assessment, January...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nKarolina Hird, Grace Mappes, Angela Howard...",\nThe Ukrainian defense of Bakhmut is likely a...,ukrainian defens bakhmut like strateg sound e...,ukrainian defen bakhmut like strateg sound ef...,"{'januari': 0.491, 'teplinski': 0.219, 'milblo..."
330,2023-01-22,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, Januar...","Russian Offensive Campaign Assessment, Januar...",/backgrounder/russian-offensive-campaign-asses...,"<div class=""field field-name-body field-type-t...","\nRussian Offensive Campaign Assessment, Janu...",\nISW is publishing an abbreviated campaign up...,isw publish abbrevi campaign updat today janu...,isw publish abbrevi campaign updat today janu...,"{'prigozhin': 0.6, 'putin': 0.328, 'wagner': 0..."
331,2023-01-23,russian_offensive_campaign_assessment_january_...,"Russian Offensive Campaign Assessment, January...","Russian Offensive Campaign Assessment, January...",/backgrounder/russian-offensive-cam

In [37]:
df.iloc[1,10]

{'pm': 0.36,
 'zero': 0.351,
 'februari': 0.336,
 'kyiv': 0.296,
 'envelop': 0.191,
 'bypass': 0.176,
 'five': 0.16,
 'local': 0.148,
 'time': 0.14,
 'enter': 0.133,
 'frontal': 0.13,
 'chernihiv': 0.125,
 'belaru': 0.122,
 'fail': 0.093,
 'cloth': 0.086,
 'advanc': 0.085,
 'stolin': 0.081,
 'may': 0.079,
 'amphibi': 0.076,
 'verg': 0.074,
 'air': 0.073,
 'next': 0.07,
 'captur': 0.069,
 'four': 0.069,
 'citi': 0.068,
 'dnipro': 0.067,
 'broad': 0.067,
 'vdv': 0.066,
 'rivn': 0.066,
 'outskirt': 0.065,
 'capabl': 0.065,
 'downtown': 0.063,
 'axi': 0.063,
 'east': 0.062,
 'kharkiv': 0.061,
 'defens': 0.061,
 'airborn': 0.061,
 'confirm': 0.061,
 'crimea': 0.059,
 'assault': 0.058,
 'hostomel': 0.056,
 'line': 0.055,
 'hour': 0.054,
 'mariupol': 0.053,
 'refrain': 0.053,
 'element': 0.053,
 'reportedli': 0.052,
 'pin': 0.052,
 'land': 0.051,
 'unconfirm': 0.051,
 'west': 0.05,
 'airport': 0.049,
 'troop': 0.049,
 'seiz': 0.048,
 'sever': 0.047,
 'yet': 0.047,
 'assembl': 0.046,
 'staff':

In [39]:
df.to_csv(f"{OUTPUT_DATA_FILE}", sep=";", index=False)